<a href="https://colab.research.google.com/github/DanParsons80/machine_learning_australian_wildlife_strike_cost_estimates/blob/main/code_ml_us_costs__to_aus_costs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminary Cells #

For importing libraries, data and other preparatory tasks

In [ ]:
### Import cell for required libraries

import pandas as pd # for data management
import numpy as np
import random
import pprint
import matplotlib.pyplot as plt
import seaborn as sns
sns.set() # Setting seaborn as default style even if use only matplotlib

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

from statsmodels.stats.weightstats import ztest

In [ ]:
### Import cell for raw data

# Australia bird strike data (01/01/2008 - 31/12/2017) downloaded from ATSB webstie (13/04/2022) - https://www.atsb.gov.au/media/5775744/ar-2018-035_birdstrike_data_table.xlsx
df_aus_data_raw = pd.read_csv('aus_data_raw.csv')

# US bird strike data downloaded from FAA website (29/05/2022) - https://widlife.faa.gov
# Unused columns removed and separated into three separate files (to allow upload to Github)
df_us_data_part_1 = pd.read_csv('us_data_concat_part_1.csv')
df_us_data_part_2 = pd.read_csv('us_data_concat_part_2.csv')
df_us_data_part_3 = pd.read_csv('us_data_concat_part_3.csv')

df_us_data_raw = pd.concat([df_us_data_part_1, df_us_data_part_2, df_us_data_part_3], ignore_index=True).drop_duplicates()

# Summary Statistics #

This section analyse's the US data to derive summary statistics for Table 1 in the paper.

In [ ]:
df_cost_data_only = pd.DataFrame()

# Need to remove the zero values to avoid -inf in log transform
df_us_data_raw.loc[df_us_data_raw['COST_REPAIRS_INFL_ADJ'] == 0, 'COST_REPAIRS_INFL_ADJ'] = np.nan
df_us_data_raw.loc[df_us_data_raw['COST_OTHER_INFL_ADJ'] == 0, 'COST_OTHER_INFL_ADJ'] = 0.01

# normal
df_cost_data_only['cost_repairs_adj'] = df_us_data_raw['COST_REPAIRS_INFL_ADJ']
df_cost_data_only['cost_other_adj'] = df_us_data_raw['COST_OTHER_INFL_ADJ']

# And now lets remove the nulls
all_repair_costs = df_cost_data_only.loc[df_cost_data_only['cost_repairs_adj'].notnull(), 'cost_repairs_adj']
all_other_costs = df_cost_data_only.loc[df_cost_data_only['cost_other_adj'].notnull(), 'cost_other_adj']

In [ ]:
df_summaries = pd.DataFrame(columns=['data_group', 'mean', 'std_dev', 'min', 'median', 'max', 'N'])

l_data_groups = [all_repair_costs, all_other_costs]
l_data_group_names = ['all_repair_costs', 'all_other_costs']

for i in range(0,len(l_data_group_names)):
  temp_dg_name = l_data_group_names[i]
  data_group = l_data_groups[i]
  temp_mean = round(data_group.mean())
  temp_std_dev = round(data_group.std())
  temp_min = round(data_group.min(), 2)
  temp_median = round(data_group.median())
  temp_max = round(data_group.max())
  temp_n = len(data_group)

  new_row = {'data_group': temp_dg_name, 'mean': temp_mean, 'std_dev': temp_std_dev, 'min': temp_min, 'median': temp_median, 'max': temp_max, 'N': temp_n}

  df_summaries.loc[len(df_summaries)] = new_row

df_summaries

,data_group,mean,std_dev,min,median,max,N
0,all_repair_costs,171491,1010921,1.00,15304,45432000,4910
1,all_other_costs,24839,187126,0.01,716,6925000,4453


# **Data Pre-processing** #

This section brings together each data set and alings them according to common scales. Primarily based on ICAO's IBIS Manual (Doc 9332).

In [ ]:
### This cell creates two dataframes for use in the ML algorithm

df_us_ml_data = pd.DataFrame() # for the US data
df_aus_ml_data = pd.DataFrame() # for the AUS data

In [ ]:
### To assist with annual and average totals at the end, let's capture the year in which each strike occurred

## US Data

df_us_ml_data['year'] = df_us_data_raw['INCIDENT_YEAR']

## AUS Data

df_aus_ml_data['year'] = df_aus_data_raw['year']

In [ ]:
### Aircraft Classification pre-processing

## This dictionary is for the aircraft class field and is based on field 0011 in the IBIS Manual (Doc 9332, 3rd Ed)
ac_dict = {'Aeroplane': 'A',
           'Helicopter': 'B',
           'Glider': 'C',
           'Balloon': 'D',
           'Dirigible': 'F',
           'Gyroplane': 'I',
           'Motor-Glider': 'J',
           'Remotely Piloted Aircraft': 'R',
           'Other': 'Y',
           'Unknown': 'Z'}

## US Data
df_us_data_raw.loc[df_us_data_raw['AC_CLASS'].isnull(), 'AC_CLASS'] = 'Z' ## The US data contains some null values which should be classed as 'Z - Unknown' as per Doc 9332
df_us_ml_data['acft_class'] = df_us_data_raw['AC_CLASS'] # now copy this column into the final ML dataframe

## AUS Data
df_aus_data_raw.loc[df_aus_data_raw['AircraftType'].isnull(), 'AircraftType'] = 'Unknown' ## The AUS data contains some null values which should be classed as 'Z - Unknown' as per Doc 9332
df_aus_ml_data['acft_class'] = df_aus_data_raw['AircraftType'].apply(lambda x: ac_dict.get(x)) # now copy this column into the final ML dataframe

In [ ]:
### Engine Type pre-processing

## This dictionary is for the type of power and is based on field 0014 in the IBIS Manual (Doc 9332, 3rd Ed)
eng_dict = {'Piston': 'A', # Reciprocating engine in Doc 9332
            'Turboprop': 'B',
            'Turbojet': 'C',
            'Turbofan': 'D',
            'Not Applicable': 'E', # None (glider) in Doc 9332
            'Turboshaft': 'F',
            'Electric': 'Y', # Other in Doc 9332
            'Unknown': 'Z'}

## US Data
df_us_data_raw.loc[df_us_data_raw['TYPE_ENG'].isnull(), 'TYPE_ENG'] = 'Z' ## The US data contains some null values which should be classed as 'Z - Unknown' as per Doc 9332
df_us_ml_data['type_eng'] = df_us_data_raw['TYPE_ENG'] # now copy this column into the final ML dataframe

## AUS Data
df_aus_data_raw.loc[df_aus_data_raw['EngineType'].isnull(), 'EngineType'] = 'Unknown' ## The AUS data contains some null values which should be classed as 'Z - Unknown' as per Doc 9332
df_aus_ml_data['type_eng'] = df_aus_data_raw['EngineType'].apply(lambda x: eng_dict.get(x)) # now copy this column into the final ML dataframe

In [ ]:
### Aircraft Mass pre-processing

## This dictionary is for the aircraft mass category and is based on field 0012 in the IBIS Manual (Doc 9332, 3rd Ed)
mass_dict = {'0-2250 Kg (0-4960 Lbs)': 1.0, # Reciprocating engine in Doc 9332
             '2251-5700 Kg (4960-12565 Lbs)': 2.0,
             '5701-27000 Kg (12565-59525 Lbs)': 3.0,
             '27001-272000 Kg (59525-599650 Lbs)': 4.0,
             'Over 272001 Kg (>599650 Lbs)': 5.0,
             'Unknown': 'Z'}

## US Data
df_us_data_raw.loc[df_us_data_raw['AC_MASS'].isnull(), 'AC_MASS'] = 'Z' ## The US data contains some null values which should be classed as 'Z - Unknown' as per Doc 9332
df_us_ml_data['mass_cat'] = df_us_data_raw['AC_MASS'] # now copy this column into the final ML dataframe


## AUS Data
df_aus_data_raw.loc[df_aus_data_raw['MaximumWeightCategory'].isnull(), 'MaximumWeightCategory'] = 'Unknown' ## The AUS data contains some null values which should be classed as 'Z - Unknown' as per Doc 9332
df_aus_ml_data['mass_cat'] = df_aus_data_raw['MaximumWeightCategory'].apply(lambda x: mass_dict.get(x)) # now copy this column into the final ML dataframe

In [ ]:
### Phase of Flight pre-processing

## This dictionary is for the phase of flight and is based on field 0117 in the IBIS Manual (Doc 9332, 3rd Ed) but a few tweaks were required because some non-standard and additional labels were used
phase_dict = {'Standing': 'A', # for non-standard label in US data
              'Parked': 'A',
              'Taxiing': 'B',
              'Taxi': 'B', # for non-standard label in US data
              'Take-off Run': 'C',
              'Takeoff': 'C', # for non-standard label in AUS data
              'Climb': 'D',
              'Inital Climb': 'D', # for additional non-standard label in AUS data
              'En Route': 'E',
              'Cruise': 'E', # for non-standard label in AUS data
              'Descent': 'F',
              'Approach': 'G',
              'Arrival': 'G', # for additional non-standard label in AUS data
              'Landing Roll': 'H',
              'Landing': 'H', # for non-standard label in AUS data
              'Maneuvering/airwork': 'I', # for unassigned label in AUS data
              'Local': 'I', # for unassigned but similar label in US data
              'None': 'Z', # for unassigned label in AUS data
              'Unknown': 'Z' } # although assigned blank in Doc 9332, this label is assigned 'Z' in this data

## US Data
df_us_data_raw.loc[df_us_data_raw['PHASE_OF_FLIGHT'].isnull(), 'PHASE_OF_FLIGHT'] = 'Unknown' ## The US data contains some null values which should be classed as 'Z - Unknown'
df_us_ml_data['phase'] = df_us_data_raw['PHASE_OF_FLIGHT'].apply(lambda x: phase_dict.get(x)) # now copy this column into the final ML dataframe

## AUS Data
df_aus_data_raw.loc[df_aus_data_raw['PhaseOfFlight'].isnull(), 'PhaseOfFlight'] = 'Unknown' ## The AUS data contains some null values which should be classed as 'Z - Unknown' as per Doc 9332
df_aus_ml_data['phase'] = df_aus_data_raw['PhaseOfFlight'].apply(lambda x: phase_dict.get(x)) # now copy this column into the final ML dataframe

In [ ]:
### Birds seen pre-processing

## This dictionary is for the birds seen and struck fields and is based on fields 0142 & 0143 in the IBIS Manual (Doc 9332, 3rd Ed)
num_dict = {'1': 'A',
             ' 1' : 'A', # unknown nbsp at start of label in US data
             ' 2-10': 'B', # unknown nbsp at start of label in US data
             '2 - 10 ': 'B', # structure of the label in AUS data
             ' 2 - 10 ': 'B', # a weird alternative in the AUS data
             ' 11-100': 'C', # unknown nbsp at start of label in US data
             '>10': 'C', # structure of the label in AUS data
             ' More than 100': 'D', # unknown nbsp at start of label in US data
             ' ': 'Z', # unknown nbsp at start of label in US data and although assigned blank in Doc 9332, this label is assigned 'Z' in this data
             'None': 'Z', # alternative label in AUS data that means the same as no value
             'Unknown': 'Z'} # although assigned blank in Doc 9332, this label is assigned 'Z' in this data

## US Data
df_us_ml_data['birds_seen'] = df_us_data_raw['NUM_SEEN'].apply(lambda x: num_dict.get(x)) # now copy this column into the final ML dataframe

## AUS Data
df_aus_data_raw.loc[df_aus_data_raw['No of birds seen'].isnull(), 'No of birds seen'] = 'Unknown' ## The AUS data contains some null values which should be classed as 'Z - Unknown' as per Doc 9332
df_aus_ml_data['birds_seen'] = df_aus_data_raw['No of birds seen'].apply(lambda x: num_dict.get(x)) # now copy this column into the final ML dataframe

In [ ]:
### Birds struck pre-processing

## This dictionary is for the birds seen and struck fields and is based on fields 0142 & 0143 in the IBIS Manual (Doc 9332, 3rd Ed)
num_dict = {'1': 'A',
             ' 1' : 'A', # unknown nbsp at start of label in US data
             ' 2-10': 'B', # unknown nbsp at start of label in US data
             '2 - 10 ': 'B', # structure of the label in AUS data
             ' 2 - 10 ': 'B', # a weird alternative in the AUS data
             ' 11-100': 'C', # unknown nbsp at start of label in US data
             '>10': 'C', # structure of the label in AUS data
             ' More than 100': 'D', # unknown nbsp at start of label in US data
             ' ': 'Z', # unknown nbsp at start of label in US data and although assigned blank in Doc 9332, this label is assigned 'Z' in this data
             'None': 'Z', # alternative label in AUS data that means the same as no value
             'Unknown': 'Z'} # although assigned blank in Doc 9332, this label is assigned 'Z' in this data

## US Data
df_us_ml_data['birds_struck'] = df_us_data_raw['NUM_STRUCK'].apply(lambda x: num_dict.get(x)) # now copy this column into the final ML dataframe

## AUS Data
df_aus_ml_data['birds_struck'] = df_aus_data_raw['No of birds struck'].apply(lambda x: num_dict.get(x)) # now copy this column into the final ML dataframe

In [ ]:
### Bird size pre-processing

## This dictionary is for the size of bird and is based on field 0144 in the IBIS Manual (Doc 9332, 3rd Ed) * Note: this is a perceived field not a scientific value
size_dict = {'small': 'S', # AUS label
            'Small': 'S', # US label
            'medium': 'M', # AUS label
            'Medium': 'M', # US label
            'large': 'L', # AUS label
            'Large': 'L', # US label
            'very large': 'L', # AUS-only label that sits outside Doc 9332
            'other': 'Z', # alternative label in AUS data that appears to carry no meaning
            'unknown': 'Z'} # although assigned blank in Doc 9332, this label is assigned 'Z' in this data

## US Data
df_us_data_raw.loc[df_us_data_raw['SIZE'].isnull(), 'SIZE'] = 'unknown' ## The US data contains some null values which should be classed as 'Z - unknown'
df_us_ml_data['bird_size'] = df_us_data_raw['SIZE'].apply(lambda x: size_dict.get(x)) # now copy this column into the final ML dataframe

## AUS Data
df_aus_ml_data['bird_size'] = df_aus_data_raw['BirdSize'].apply(lambda x: size_dict.get(x)) # now copy this column into the final ML dataframe

In [ ]:
### Aircraft damage pre-prepocessing

## This dictionary is based on field 0201 in the IBIS Manual (Doc 9332, 3rd Ed) * Note: additional field for uncertain/unknown damage retained from US data
damage_dict = {'Nil ': 'N',
            'Minor': 'M',
            'Unknown': 'M?', # US label retained with AUS unknown considered equivalent as nil appears to be null
            'Substantial': 'S',
            'Destroyed': 'D',
            'Missing': 'Z'} # although assigned blank in Doc 9332, this label is assigned 'Z' in this data

## US Data
df_us_data_raw.loc[df_us_data_raw['DAMAGE_LEVEL'].isnull(), 'DAMAGE_LEVEL'] = 'Z' ## The US data contains some null values which should be classed as 'Z - Missing'
df_us_data_raw.loc[((df_us_data_raw['DAMAGE_LEVEL'] == 'N') & (df_us_data_raw['COST_REPAIRS_INFL_ADJ'].notnull())), 'DAMAGE_LEVEL'] = 'M?' # There are some repair costs records with damage level set at nil this needs to be fixed
df_us_data_raw.loc[((df_us_data_raw['DAMAGE_LEVEL'] == 'Z') & (df_us_data_raw['COST_REPAIRS_INFL_ADJ'].notnull())), 'DAMAGE_LEVEL'] = 'M?' # There are some repair costs records with damage level set at missing this needs to be fixed
df_us_ml_data['damage_level'] = df_us_data_raw['DAMAGE_LEVEL'] # now copy this column into the final ML dataframe

## AUS Data
df_aus_data_raw.loc[df_aus_data_raw['AircraftDamageLevel'].isnull(), 'AircraftDamageLevel'] = 'Missing' ## The AUS data contains some null values which should be classed as 'Z - Missing'
df_aus_ml_data['damage_level'] = df_aus_data_raw['AircraftDamageLevel'].apply(lambda x: damage_dict.get(x)) # now copy this column into the final ML dataframe

In [ ]:
### Ingestion pre-processing

## This dictionary is based on the binary nature of much of the US data. Doc 9332 expects the numbers of birds ingested in each engine to be recorded but neither the AUS or US data does this.
ing_dict = {'No': 0,
            'Unknown': 0,
            'Yes': 1,
            '1 Engine': 1,
            '2 Engines': 1}

## US Data (doesn't need the dictionary above)
df_us_data_raw.loc[df_us_data_raw['INGESTED_OTHER'] == True, 'TEMP_ING'] = 1 # This field was used to capture all ingested data prior to 29/3/2021.
df_us_data_raw.loc[df_us_data_raw['ING_ENG1'] == True, 'TEMP_ING'] = 1 # This and the following fields were used to capture whether ingestions were recorded in each engine from 29/3/2021
df_us_data_raw.loc[df_us_data_raw['ING_ENG2'] == True, 'TEMP_ING'] = 1
df_us_data_raw.loc[df_us_data_raw['ING_ENG3'] == True, 'TEMP_ING'] = 1
df_us_data_raw.loc[df_us_data_raw['ING_ENG4'] == True, 'TEMP_ING'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_ING'].isnull(), 'TEMP_ING'] = 0
df_us_ml_data['ingestion'] = df_us_data_raw['TEMP_ING'] # now copy this column into the final ML dataframe
df_us_ml_data['ingestion'] = df_us_ml_data['ingestion'].astype(int)

## AUS Data
df_aus_data_raw.loc[df_aus_data_raw['Engine Ingestion'].isnull(), 'Engine Ingestion'] = 'Unknown'
df_aus_ml_data['ingestion'] = df_aus_data_raw['Engine Ingestion'].apply(lambda x: ing_dict.get(x)) # now copy this column into the final ML dataframe
df_aus_ml_data['ingestion'] = df_aus_ml_data['ingestion'].astype(int)

In [ ]:
### Component damage pre-processing

## This one needs signficiant work up front as it begins as one-hot coded, dummy variables in the US data (albeit as true/false)

## US Data
# Radome & Nose Separate - for use in testing this ML approach against the established method
df_us_data_raw.loc[df_us_data_raw['DAM_RAD'] == True, 'TEMP_RAD'] = 1
df_us_data_raw.loc[df_us_data_raw['DAM_NOSE'] == True, 'TEMP_NOSE'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_RAD'].isnull(), 'TEMP_RAD'] = 0
df_us_data_raw.loc[df_us_data_raw['TEMP_NOSE'].isnull(), 'TEMP_NOSE'] = 0
df_us_ml_data['comp_dam_rad'] = df_us_data_raw['TEMP_RAD'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_dam_rad'] = df_us_ml_data['comp_dam_rad'].astype(int)
df_us_ml_data['comp_dam_nose'] = df_us_data_raw['TEMP_NOSE'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_dam_nose'] = df_us_ml_data['comp_dam_nose'].astype(int)

# Radome & Nose Combined - for use in comparing to AUS data which does not have Radome as a separate option
df_us_data_raw.loc[df_us_data_raw['DAM_RAD'] == True, 'TEMP_RAD_NOSE'] = 1
df_us_data_raw.loc[df_us_data_raw['DAM_NOSE'] == True, 'TEMP_RAD_NOSE'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_RAD_NOSE'].isnull(), 'TEMP_RAD_NOSE'] = 0
df_us_ml_data['comp_dam_rad_nose'] = df_us_data_raw['TEMP_RAD_NOSE'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_dam_rad_nose'] = df_us_ml_data['comp_dam_rad_nose'].astype(int)

# Windshield
df_us_data_raw.loc[df_us_data_raw['DAM_WINDSHLD'] == True, 'TEMP_WSD'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_WSD'].isnull(), 'TEMP_WSD'] = 0
df_us_ml_data['comp_dam_wind'] = df_us_data_raw['TEMP_WSD'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_dam_wind'] = df_us_ml_data['comp_dam_wind'].astype(int)

# Engines separate
df_us_data_raw.loc[df_us_data_raw['DAM_ENG1'] == True, 'TEMP_ENG1'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_ENG1'].isnull(), 'TEMP_ENG1'] = 0
df_us_ml_data['comp_dam_eng_1'] = df_us_data_raw['TEMP_ENG1'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_dam_eng_1'] = df_us_ml_data['comp_dam_eng_1'].astype(int)
df_us_data_raw.loc[df_us_data_raw['DAM_ENG2'] == True, 'TEMP_ENG2'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_ENG2'].isnull(), 'TEMP_ENG2'] = 0
df_us_ml_data['comp_dam_eng_2'] = df_us_data_raw['TEMP_ENG2'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_dam_eng_2'] = df_us_ml_data['comp_dam_eng_2'].astype(int)
df_us_data_raw.loc[df_us_data_raw['DAM_ENG3'] == True, 'TEMP_ENG3'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_ENG3'].isnull(), 'TEMP_ENG3'] = 0
df_us_ml_data['comp_dam_eng_3'] = df_us_data_raw['TEMP_ENG3'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_dam_eng_3'] = df_us_ml_data['comp_dam_eng_3'].astype(int)
df_us_data_raw.loc[df_us_data_raw['DAM_ENG4'] == True, 'TEMP_ENG4'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_ENG4'].isnull(), 'TEMP_ENG4'] = 0
df_us_ml_data['comp_dam_eng_4'] = df_us_data_raw['TEMP_ENG4'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_dam_eng_4'] = df_us_ml_data['comp_dam_eng_4'].astype(int)

# Engines combined - US data identifies each engine but AUS data combines them
df_us_data_raw.loc[df_us_data_raw['DAM_ENG1'] == True, 'TEMP_ENG_ALL'] = 1
df_us_data_raw.loc[df_us_data_raw['DAM_ENG2'] == True, 'TEMP_ENG_ALL'] = 1
df_us_data_raw.loc[df_us_data_raw['DAM_ENG3'] == True, 'TEMP_ENG_ALL'] = 1
df_us_data_raw.loc[df_us_data_raw['DAM_ENG4'] == True, 'TEMP_ENG_ALL'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_ENG_ALL'].isnull(), 'TEMP_ENG_ALL'] = 0
df_us_ml_data['comp_dam_eng_all'] = df_us_data_raw['TEMP_ENG_ALL'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_dam_eng_all'] = df_us_ml_data['comp_dam_eng_all'].astype(int)

# Propellers
df_us_data_raw.loc[df_us_data_raw['DAM_PROP'] == True, 'TEMP_PROP'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_PROP'].isnull(), 'TEMP_PROP'] = 0
df_us_ml_data['comp_dam_prop'] = df_us_data_raw['TEMP_PROP'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_dam_prop'] = df_us_ml_data['comp_dam_prop'].astype(int)

# Wings & Rotors
df_us_data_raw.loc[df_us_data_raw['DAM_WING_ROT'] == True, 'TEMP_WING'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_WING'].isnull(), 'TEMP_WING'] = 0
df_us_ml_data['comp_dam_wing_rotor'] = df_us_data_raw['TEMP_WING'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_dam_wing_rotor'] = df_us_ml_data['comp_dam_wing_rotor'].astype(int)

# Fuselage
df_us_data_raw.loc[df_us_data_raw['DAM_FUSE'] == True, 'TEMP_FUSE'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_FUSE'].isnull(), 'TEMP_FUSE'] = 0
df_us_ml_data['comp_dam_fuselage'] = df_us_data_raw['TEMP_FUSE'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_dam_fuselage'] = df_us_ml_data['comp_dam_fuselage'].astype(int)

# Tail
df_us_data_raw.loc[df_us_data_raw['DAM_TAIL'] == True, 'TEMP_TAIL'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_TAIL'].isnull(), 'TEMP_TAIL'] = 0
df_us_ml_data['comp_dam_tail'] = df_us_data_raw['TEMP_TAIL'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_dam_tail'] = df_us_ml_data['comp_dam_tail'].astype(int)

# Landing Gear
df_us_data_raw.loc[df_us_data_raw['DAM_LG'] == True, 'TEMP_LG'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_LG'].isnull(), 'TEMP_LG'] = 0
df_us_ml_data['comp_dam_gear'] = df_us_data_raw['TEMP_LG'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_dam_gear'] = df_us_ml_data['comp_dam_gear'].astype(int)

# Lights
df_us_data_raw.loc[df_us_data_raw['DAM_LGHTS'] == True, 'TEMP_LGHTS'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_LGHTS'].isnull(), 'TEMP_LGHTS'] = 0
df_us_ml_data['comp_dam_lights'] = df_us_data_raw['TEMP_LGHTS'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_dam_lights'] = df_us_ml_data['comp_dam_lights'].astype(int)

# Other
df_us_data_raw.loc[df_us_data_raw['DAM_OTHER'] == True, 'TEMP_OTHER'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_OTHER'].isnull(), 'TEMP_OTHER'] = 0
df_us_ml_data['comp_dam_other'] = df_us_data_raw['TEMP_OTHER'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_dam_other'] = df_us_ml_data['comp_dam_other'].astype(int)

## AUS Data
# Nose - assuming that is includes radome
df_aus_data_raw.loc[df_aus_data_raw['Part damaged'] == 'Nose', 'temp_nose'] = 1
df_aus_data_raw.loc[df_aus_data_raw['temp_nose'].isnull(), 'temp_nose'] = 0
df_aus_ml_data['comp_dam_rad_nose'] = df_aus_data_raw['temp_nose'] # now copy this column into the final ML dataframe
df_aus_ml_data['comp_dam_rad_nose'] = df_aus_ml_data['comp_dam_rad_nose'].astype(int)

# Windshield
df_aus_data_raw.loc[df_aus_data_raw['Part damaged'] == 'Windscreen', 'temp_screen'] = 1
df_aus_data_raw.loc[df_aus_data_raw['Part damaged'] == 'windscreen', 'temp_screen'] = 1
df_aus_data_raw.loc[df_aus_data_raw['temp_screen'].isnull(), 'temp_screen'] = 0
df_aus_ml_data['comp_dam_wind'] = df_aus_data_raw['temp_screen'] # now copy this column into the final ML dataframe
df_aus_ml_data['comp_dam_wind'] = df_aus_ml_data['comp_dam_wind'].astype(int)

# Engines - US data identifies each engine but AUS data combines them
df_aus_data_raw.loc[df_aus_data_raw['Part damaged'] == 'Engine', 'temp_eng'] = 1
df_aus_data_raw.loc[df_aus_data_raw['temp_eng'].isnull(), 'temp_eng'] = 0
df_aus_ml_data['comp_dam_eng_all'] = df_aus_data_raw['temp_eng'] # now copy this column into the final ML dataframe
df_aus_ml_data['comp_dam_eng_all'] = df_aus_ml_data['comp_dam_eng_all'].astype(int)

# Propellers
df_aus_data_raw.loc[df_aus_data_raw['Part damaged'] == 'Propeller', 'temp_prop'] = 1
df_aus_data_raw.loc[df_aus_data_raw['temp_prop'].isnull(), 'temp_prop'] = 0
df_aus_ml_data['comp_dam_prop'] = df_aus_data_raw['temp_prop'] # now copy this column into the final ML dataframe
df_aus_ml_data['comp_dam_prop'] = df_aus_ml_data['comp_dam_prop'].astype(int)

# Wings & Rotors
df_aus_data_raw.loc[df_aus_data_raw['Part damaged'] == 'Wing/Rotor', 'temp_wing'] = 1
df_aus_data_raw.loc[df_aus_data_raw['Part damaged'] == 'Wing/rotor', 'temp_wing'] = 1
df_aus_data_raw.loc[df_aus_data_raw['temp_wing'].isnull(), 'temp_wing'] = 0
df_aus_ml_data['comp_dam_wing_rotor'] = df_aus_data_raw['temp_wing'] # now copy this column into the final ML dataframe
df_aus_ml_data['comp_dam_wing_rotor'] = df_aus_ml_data['comp_dam_wing_rotor'].astype(int)

# Fuselage
df_aus_data_raw.loc[df_aus_data_raw['Part damaged'] == 'Fuselage', 'temp_fuselage'] = 1
df_aus_data_raw.loc[df_aus_data_raw['temp_fuselage'].isnull(), 'temp_fuselage'] = 0
df_aus_ml_data['comp_dam_fuselage'] = df_aus_data_raw['temp_fuselage'] # now copy this column into the final ML dataframe
df_aus_ml_data['comp_dam_fuselage'] = df_aus_ml_data['comp_dam_fuselage'].astype(int)

# Tail
df_aus_data_raw.loc[df_aus_data_raw['Part damaged'] == 'Tail', 'temp_tail'] = 1
df_aus_data_raw.loc[df_aus_data_raw['temp_tail'].isnull(), 'temp_tail'] = 0
df_aus_ml_data['comp_dam_tail'] = df_aus_data_raw['temp_tail'] # now copy this column into the final ML dataframe
df_aus_ml_data['comp_dam_tail'] = df_aus_ml_data['comp_dam_tail'].astype(int)

# Landing Gear
df_aus_data_raw.loc[df_aus_data_raw['Part damaged'] == 'Landing gear', 'temp_gear'] = 1
df_aus_data_raw.loc[df_aus_data_raw['temp_gear'].isnull(), 'temp_gear'] = 0
df_aus_ml_data['comp_dam_gear'] = df_aus_data_raw['temp_gear'] # now copy this column into the final ML dataframe
df_aus_ml_data['comp_dam_gear'] = df_aus_ml_data['comp_dam_gear'].astype(int)

# Lights
df_aus_data_raw.loc[df_aus_data_raw['Part damaged'] == 'Lights', 'temp_lights'] = 1
df_aus_data_raw.loc[df_aus_data_raw['temp_lights'].isnull(), 'temp_lights'] = 0
df_aus_ml_data['comp_dam_lights'] = df_aus_data_raw['temp_lights'] # now copy this column into the final ML dataframe
df_aus_ml_data['comp_dam_lights'] = df_aus_ml_data['comp_dam_lights'].astype(int)

# Other
df_aus_data_raw.loc[df_aus_data_raw['Part damaged'] == 'Other', 'temp_other'] = 1
df_aus_data_raw.loc[df_aus_data_raw['Part damaged'] == 'Unknown', 'temp_other'] = 1
df_aus_data_raw.loc[df_aus_data_raw['Part damaged'] == 'unknown', 'temp_other'] = 1
df_aus_data_raw.loc[df_aus_data_raw['temp_other'].isnull(), 'temp_other'] = 0
df_aus_ml_data['comp_dam_other'] = df_aus_data_raw['temp_other'] # now copy this column into the final ML dataframe
df_aus_ml_data['comp_dam_other'] = df_aus_ml_data['comp_dam_other'].astype(int)

In [ ]:
### Component struck pre-processing

## This is just for the US data in the testing of the ML technique without this data

## US Data
# Radome
df_us_data_raw.loc[df_us_data_raw['STR_RAD'] == True, 'TEMP_STR_RAD'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_STR_RAD'].isnull(), 'TEMP_STR_RAD'] = 0
df_us_ml_data['comp_str_rad'] = df_us_data_raw['TEMP_STR_RAD'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_str_rad'] = df_us_ml_data['comp_str_rad'].astype(int)

# Nose
df_us_data_raw.loc[df_us_data_raw['STR_NOSE'] == True, 'TEMP_STR_NOSE'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_STR_NOSE'].isnull(), 'TEMP_STR_NOSE'] = 0
df_us_ml_data['comp_str_nose'] = df_us_data_raw['TEMP_STR_NOSE'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_str_nose'] = df_us_ml_data['comp_str_nose'].astype(int)

# Windshield
df_us_data_raw.loc[df_us_data_raw['STR_WINDSHLD'] == True, 'TEMP_STR_WSD'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_STR_WSD'].isnull(), 'TEMP_STR_WSD'] = 0
df_us_ml_data['comp_str_wind'] = df_us_data_raw['TEMP_STR_WSD'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_str_wind'] = df_us_ml_data['comp_str_wind'].astype(int)

# Engines separate
df_us_data_raw.loc[df_us_data_raw['STR_ENG1'] == True, 'TEMP_STR_ENG1'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_STR_ENG1'].isnull(), 'TEMP_STR_ENG1'] = 0
df_us_ml_data['comp_str_eng_1'] = df_us_data_raw['TEMP_STR_ENG1'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_str_eng_1'] = df_us_ml_data['comp_str_eng_1'].astype(int)
df_us_data_raw.loc[df_us_data_raw['STR_ENG2'] == True, 'TEMP_STR_ENG2'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_STR_ENG2'].isnull(), 'TEMP_STR_ENG2'] = 0
df_us_ml_data['comp_str_eng_2'] = df_us_data_raw['TEMP_STR_ENG2'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_str_eng_2'] = df_us_ml_data['comp_str_eng_2'].astype(int)
df_us_data_raw.loc[df_us_data_raw['STR_ENG3'] == True, 'TEMP_STR_ENG3'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_STR_ENG3'].isnull(), 'TEMP_STR_ENG3'] = 0
df_us_ml_data['comp_str_eng_3'] = df_us_data_raw['TEMP_STR_ENG3'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_str_eng_3'] = df_us_ml_data['comp_str_eng_3'].astype(int)
df_us_data_raw.loc[df_us_data_raw['STR_ENG4'] == True, 'TEMP_STR_ENG4'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_STR_ENG4'].isnull(), 'TEMP_STR_ENG4'] = 0
df_us_ml_data['comp_str_eng_4'] = df_us_data_raw['TEMP_STR_ENG4'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_str_eng_4'] = df_us_ml_data['comp_str_eng_4'].astype(int)

# Propellers
df_us_data_raw.loc[df_us_data_raw['STR_PROP'] == True, 'TEMP_STR_PROP'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_STR_PROP'].isnull(), 'TEMP_STR_PROP'] = 0
df_us_ml_data['comp_str_prop'] = df_us_data_raw['TEMP_STR_PROP'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_str_prop'] = df_us_ml_data['comp_str_prop'].astype(int)

# Wings & Rotors
df_us_data_raw.loc[df_us_data_raw['STR_WING_ROT'] == True, 'TEMP_STR_WING'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_STR_WING'].isnull(), 'TEMP_STR_WING'] = 0
df_us_ml_data['comp_str_wing_rotor'] = df_us_data_raw['TEMP_STR_WING'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_str_wing_rotor'] = df_us_ml_data['comp_str_wing_rotor'].astype(int)

# Fuselage
df_us_data_raw.loc[df_us_data_raw['STR_FUSE'] == True, 'TEMP_STR_FUSE'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_STR_FUSE'].isnull(), 'TEMP_STR_FUSE'] = 0
df_us_ml_data['comp_str_fuselage'] = df_us_data_raw['TEMP_STR_FUSE'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_str_fuselage'] = df_us_ml_data['comp_str_fuselage'].astype(int)

# Tail
df_us_data_raw.loc[df_us_data_raw['STR_TAIL'] == True, 'TEMP_STR_TAIL'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_STR_TAIL'].isnull(), 'TEMP_STR_TAIL'] = 0
df_us_ml_data['comp_str_tail'] = df_us_data_raw['TEMP_STR_TAIL'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_str_tail'] = df_us_ml_data['comp_str_tail'].astype(int)

# Landing Gear
df_us_data_raw.loc[df_us_data_raw['STR_LG'] == True, 'TEMP_STR_LG'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_STR_LG'].isnull(), 'TEMP_STR_LG'] = 0
df_us_ml_data['comp_str_gear'] = df_us_data_raw['TEMP_STR_LG'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_str_gear'] = df_us_ml_data['comp_str_gear'].astype(int)

# Lights
df_us_data_raw.loc[df_us_data_raw['STR_LGHTS'] == True, 'TEMP_STR_LGHTS'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_STR_LGHTS'].isnull(), 'TEMP_STR_LGHTS'] = 0
df_us_ml_data['comp_str_lights'] = df_us_data_raw['TEMP_STR_LGHTS'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_str_lights'] = df_us_ml_data['comp_str_lights'].astype(int)

# Other
df_us_data_raw.loc[df_us_data_raw['STR_OTHER'] == True, 'TEMP_STR_OTHER'] = 1
df_us_data_raw.loc[df_us_data_raw['TEMP_STR_OTHER'].isnull(), 'TEMP_STR_OTHER'] = 0
df_us_ml_data['comp_str_other'] = df_us_data_raw['TEMP_STR_OTHER'] # now copy this column into the final ML dataframe
df_us_ml_data['comp_str_other'] = df_us_ml_data['comp_str_other'].astype(int)

In [ ]:
### Other items for pre-processing in the US Data only

## The following fields are not found in the AUS data but to provide an indication of the ML algorithm's validity when limited to AUS data fields we need to test both

## US Data

# Pilot warned
## This dictionary is based on field 0145 in the IBIS Manual (Doc 9332, 3rd Ed)
warned_dict = {'Yes': 'Y',
               'No': 'N',
               'Unknown': 'Z'} # although assigned blank in Doc 9332, this label is assigned 'Z' in this data

df_us_data_raw.loc[df_us_data_raw['WARNED'].isnull(), 'WARNED'] = 'Unknown' # need to reassign null values to capture in standardised format
df_us_ml_data['warned'] = df_us_data_raw['WARNED'].apply(lambda x: warned_dict.get(x)) # now copy this column into the final ML dataframe

# Effect on flight
## This feature is quoted as a categorical value in Altringer et al. (2021) but should be one hot coded bcasue some fields contain mutiple categories

df_us_ml_data['effect_nil'] = 0 # let's set them all to 0 first
df_temp_effect_nil = df_us_data_raw[df_us_data_raw['EFFECT'] == ' None'] # then get the indexes for all the records that say 'None' only
df_us_ml_data.loc[df_temp_effect_nil.index, 'effect_nil'] = 1 # and then we switch the records with this value to 1

df_us_ml_data['effect_pre_landing'] = 0 # let's set them all to 0 first
df_temp_effect_pre_landing = df_us_data_raw[df_us_data_raw['EFFECT'].str.contains('Precautionary Landing') == True] # then get the indexes for all the records that including this text in them
df_us_ml_data.loc[df_temp_effect_pre_landing.index, 'effect_pre_landing'] = 1 # and then we switch the records with this value to 1

df_us_ml_data['effect_abort_to'] = 0 # let's set them all to 0 first
df_temp_effect_abort_to = df_us_data_raw[df_us_data_raw['EFFECT'].str.contains('Aborted Take-off') == True] # then get the indexes for all the records that including this text in them
df_us_ml_data.loc[df_temp_effect_abort_to.index, 'effect_abort_to'] = 1 # and then we switch the records with this value to 1

df_us_ml_data['effect_eng_sd'] = 0 # let's set them all to 0 first
df_temp_effect_eng_sd = df_us_data_raw[df_us_data_raw['EFFECT'].str.contains('Engine Shutdown') == True] # then get the indexes for all the records that including this text in them
df_us_ml_data.loc[df_temp_effect_eng_sd.index, 'effect_eng_sd'] = 1 # and then we switch the records with this value to 1

df_us_ml_data['effect_other'] = 0 # let's set them all to 0 first
df_temp_effect_other = df_us_data_raw[df_us_data_raw['EFFECT'].str.contains('Other') == True] # then get the indexes for all the records that including this text in them
df_us_ml_data.loc[df_temp_effect_other.index, 'effect_other'] = 1 # and then we switch the records with this value to 1

# Cloud Cover
## This dictionary is based on field 0137 in the IBIS Manual (Doc 9332, 3rd Ed)
sky_dict = {'No Cloud': 'A',
               'Some Cloud': 'B',
               'Overcast': 'C',
               'Unknown': 'Z'} # although assigned blank in Doc 9332, this label is assigned 'Z' in this data

df_us_data_raw.loc[df_us_data_raw['SKY'].isnull(), 'SKY'] = 'Unknown' # need to reassign null values to capture in standardised format
df_us_ml_data['sky'] = df_us_data_raw['SKY'].apply(lambda x: sky_dict.get(x)) # now copy this column into the final ML dataframe

# Time of Day
## This dictionary is based on field 0110 in the IBIS Manual (Doc 9332, 3rd Ed)
light_dict = {'Dawn': 'A',
              'Day': 'B',
              'Dusk': 'C',
              'Night': 'D',
              'Unknown': 'Z'} # although assigned blank in Doc 9332, this label is assigned 'Z' in this data

df_us_data_raw.loc[df_us_data_raw['TIME_OF_DAY'].isnull(), 'TIME_OF_DAY'] = 'Unknown' # need to reassign null values to capture in standardised format
df_us_ml_data['light_conditions'] = df_us_data_raw['TIME_OF_DAY'].apply(lambda x: light_dict.get(x)) # now copy this column into the final ML dataframe


In [ ]:
### And now the target variables

## US Data Only
# Need to remove the zero values to avoid -inf in log transform
df_us_data_raw.loc[df_us_data_raw['COST_REPAIRS_INFL_ADJ'] == 0, 'COST_REPAIRS_INFL_ADJ'] = np.nan
df_us_data_raw.loc[df_us_data_raw['COST_OTHER_INFL_ADJ'] == 0, 'COST_OTHER_INFL_ADJ'] = 0.01

# normal
df_us_ml_data['cost_repairs_adj'] = df_us_data_raw['COST_REPAIRS_INFL_ADJ']
df_us_ml_data['cost_other_adj'] = df_us_data_raw['COST_OTHER_INFL_ADJ']

# log transformed
df_us_ml_data['log_cost_repairs_adj'] = np.log(df_us_data_raw['COST_REPAIRS_INFL_ADJ'])
df_us_ml_data['log_cost_other_adj'] = np.log(df_us_data_raw['COST_OTHER_INFL_ADJ'])

In [ ]:
### Finally, let's now create our datasets for use in the following model development sections

#### This cell is for full-one hot encoding ####

## US Data

# The first step is to filter for reports according to damage
# Only reports indicating damage are included in the damage cost model

l_damage_inc = ['M', 'M?', 'S', 'D']

df_us_ml_damaged_inc = df_us_ml_data[df_us_ml_data['damage_level'].isin(l_damage_inc)]

# The following creates the required sub-sets for replication of the Altringer et al (2021) RF modelling technique

# From these sub-sets we need to further filter for only those records with reported costs

df_repair_inc = df_us_ml_damaged_inc[df_us_ml_damaged_inc['log_cost_repairs_adj'].notnull()]
df_other_inc = df_us_ml_data[df_us_ml_data['log_cost_other_adj'].notnull()]

# And now we need dummy variables (OHE) for all of our categorical data

# Replication Structure - Repair Costs
X_rep_repair_inc = pd.get_dummies(df_repair_inc, prefix_sep='_') # OHE for categorical variables
X_rep_repair_inc = X_rep_repair_inc.drop(['year', 'comp_dam_rad_nose', 'cost_repairs_adj', 'cost_other_adj', 'log_cost_repairs_adj', 'log_cost_other_adj'], axis=1) # dropping target and other variables
y_rep_repair_inc = df_repair_inc['log_cost_repairs_adj']

# Replication Structure - Other Costs
X_rep_other_inc = pd.get_dummies(df_other_inc, prefix_sep='_') # OHC for categorical variables
X_rep_other_inc = X_rep_other_inc.drop(['year', 'comp_dam_rad_nose', 'cost_repairs_adj', 'cost_other_adj', 'log_cost_repairs_adj', 'log_cost_other_adj'], axis=1) # dropping target and other variables
y_rep_other_inc = df_other_inc['log_cost_other_adj']


# Constrained Structure set up

# These features are not included in the AUS data
l_drop_features = ['comp_dam_rad', 'comp_dam_nose', 'comp_dam_eng_1', 'comp_dam_eng_2', 'comp_dam_eng_3', 'comp_dam_eng_4',
                  'comp_str_rad', 'comp_str_nose','comp_str_wind', 'comp_str_eng_1', 'comp_str_eng_2', 'comp_str_eng_3', 'comp_str_eng_4',
                  'comp_str_prop', 'comp_str_wing_rotor', 'comp_str_fuselage', 'comp_str_tail', 'comp_str_gear', 'comp_str_lights',
                  'comp_str_other','warned', 'effect_nil', 'effect_pre_landing', 'effect_abort_to', 'effect_eng_sd', 'effect_other', 'sky', 'light_conditions']

# These variables (feature categories) are not included in the AUS data
l_drop_variables = ['type_eng_C', 'birds_seen_D', 'birds_struck_D']

# Constrained Structure - Repair Costs
X_cons_repair_inc = df_repair_inc.drop(l_drop_features, axis=1)
X_cons_repair_inc = pd.get_dummies(X_cons_repair_inc, prefix_sep='_')
X_cons_repair_inc = X_cons_repair_inc.drop(['year', 'cost_repairs_adj', 'cost_other_adj', 'log_cost_repairs_adj', 'log_cost_other_adj'], axis=1)
X_cons_repair_inc.columns = X_cons_repair_inc.columns.str.replace('  ', '')
X_cons_repair_inc = X_cons_repair_inc.drop(l_drop_variables, axis=1)
y_cons_repair_inc = df_repair_inc['log_cost_repairs_adj']

# Constrained Structure - Other Costs
X_cons_other_inc = df_other_inc.drop(l_drop_features, axis=1)
X_cons_other_inc = pd.get_dummies(X_cons_other_inc, prefix_sep='_')
X_cons_other_inc = X_cons_other_inc.drop(['year', 'cost_repairs_adj', 'cost_other_adj', 'log_cost_repairs_adj', 'log_cost_other_adj'], axis=1)
X_cons_other_inc.columns = X_cons_other_inc.columns.str.replace('  ', '')
X_cons_other_inc = X_cons_other_inc.drop(l_drop_variables, axis=1)
y_cons_other_inc = df_other_inc['log_cost_other_adj']


## And now we need to segregate into training & test samples (80/20)

X_rep_repair_inc_train, X_rep_repair_inc_test, y_rep_repair_inc_train, y_rep_repair_inc_test = train_test_split(X_rep_repair_inc, y_rep_repair_inc, test_size=0.2, random_state=23)
X_rep_other_inc_train, X_rep_other_inc_test, y_rep_other_inc_train, y_rep_other_inc_test = train_test_split(X_rep_other_inc, y_rep_other_inc, test_size=0.2, random_state=23)

X_cons_repair_inc_train, X_cons_repair_inc_test, y_cons_repair_inc_train, y_cons_repair_inc_test = train_test_split(X_cons_repair_inc, y_cons_repair_inc, test_size=0.2, random_state=23)
X_cons_other_inc_train, X_cons_other_inc_test, y_cons_other_inc_train, y_cons_other_inc_test = train_test_split(X_cons_other_inc, y_cons_other_inc, test_size=0.2, random_state=23)


# **Machine Learning Preparation** #

## Preliminary Cells ##

This section includes cells that are used repeatedly in each section below

In [ ]:
### This is a randomised grid for use in model development

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1800, num = 9)] # Number of trees in random forest
max_features = ['sqrt'] # Number of features to consider at every split
max_features.append(None)
criterion = ['squared_error']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)] # Maximum number of levels in tree
max_depth.append(None)
min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 10, num = 9)] # Minimum number of samples required to split a node
min_samples_leaf = [1, 2, 4] # Minimum number of samples required at each leaf node
bootstrap = [True, False] # Method of selecting samples for training each tree

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'criterion': criterion,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint.pprint(random_grid) # Just for an overview

{'bootstrap': [True, False],
 'criterion': ['squared_error'],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['sqrt', None],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800]}


In [ ]:
rf = RandomForestRegressor()

## Random Search CV ##

These cells conduct a randomised search for the best model hyperparameters using cross-validation.

In [ ]:
## Replication - Repair Costs

rf_random_rep_repair_inc = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 10, verbose=1, random_state=23, n_jobs = -1)
rf_random_rep_repair_inc.fit(X_rep_repair_inc_train, y_rep_repair_inc_train)

rf_random_rep_repair_inc.best_params_ # This returns the best parameters for use in the refined grid search

In [ ]:
## Replication - Other Costs

rf_random_rep_other_inc = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 10, verbose=1, random_state=23, n_jobs = -1)
rf_random_rep_other_inc.fit(X_rep_other_inc_train, y_rep_other_inc_train)

rf_random_rep_other_inc.best_params_ # This returns the best parameters for use in the refined grid search

In [ ]:
## Constrained - Repair Costs

rf_random_cons_repair_inc = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 10, verbose=1, random_state=23, n_jobs = -1)
rf_random_cons_repair_inc.fit(X_cons_repair_inc_train, y_cons_repair_inc_train)

rf_random_cons_repair_inc.best_params_ # This returns the best parameters for use in the refined grid search

In [ ]:
## Constrained - Other Costs

rf_random_cons_other_inc = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 10, verbose=1, random_state=23, n_jobs = -1)
rf_random_cons_other_inc.fit(X_cons_other_inc_train, y_cons_other_inc_train)

rf_random_cons_other_inc.best_params_ # This returns the best parameters for use in the refined grid search

## Grid Search CV ##

These cells undertake a more methodical grid search for the best hyperparameters using those found in the randomised search.

In [ ]:
## More refined grid search with cross validation

## based on rf_random's best parameters
refined_grid_rep_repair_inc = {'n_estimators': [1700, 1800, 1900, 2000],
                               'max_features': ['sqrt'],
                               'criterion': ['squared_error'],
                               'max_depth': [40],
                               'min_samples_split': [6],
                               'min_samples_leaf': [2],
                               'bootstrap': [False]}

rf_grid_rep_repair_inc = GridSearchCV(estimator = rf, param_grid = refined_grid_rep_repair_inc, cv = 10, n_jobs = -1, verbose = 2)

rf_grid_rep_repair_inc.fit(X_rep_repair_inc_train, y_rep_repair_inc_train)

rf_grid_rep_repair_inc.best_params_ # This returns the best parameters for use in the final model

In [ ]:
## More refined grid search with cross validation

## based on rf_random's best parameters
refined_grid_rep_other_inc = {'n_estimators': [1500, 1600, 1700],
                               'max_features': ['sqrt'],
                               'criterion': ['squared_error'],
                               'max_depth': [20],
                               'min_samples_split': [8],
                               'min_samples_leaf': [2],
                               'bootstrap': [False]}

rf_grid_rep_other_inc = GridSearchCV(estimator = rf, param_grid = refined_grid_rep_other_inc, cv = 10, n_jobs = -1, verbose = 2)

rf_grid_rep_other_inc.fit(X_rep_other_inc_train, y_rep_other_inc_train)

rf_grid_rep_other_inc.best_params_ # This returns the best parameters for use in the final model

In [ ]:
## More refined grid search with cross validation

## based on rf_random's best parameters
refined_grid_cons_repair_inc = {'n_estimators': [500, 600, 700],
                               'max_features': ['sqrt'],
                               'criterion': ['squared_error'],
                               'max_depth': [40],
                               'min_samples_split': [5],
                               'min_samples_leaf': [2],
                               'bootstrap': [False]}

rf_grid_cons_repair_inc = GridSearchCV(estimator = rf, param_grid = refined_grid_cons_repair_inc, cv = 10, n_jobs = -1, verbose = 2)

rf_grid_cons_repair_inc.fit(X_cons_repair_inc_train, y_cons_repair_inc_train)

rf_grid_cons_repair_inc.best_params_ # This returns the best parameters for use in the final model

In [ ]:
## More refined grid search with cross validation

## based on rf_random's best parameters
refined_grid_cons_other_inc = {'n_estimators': [900],
                               'max_features': ['sqrt'],
                               'criterion': ['squared_error'],
                               'max_depth': [130, 150, 170],
                               'min_samples_split': [5],
                               'min_samples_leaf': [4],
                               'bootstrap': [False]}

rf_grid_cons_other_inc = GridSearchCV(estimator = rf, param_grid = refined_grid_cons_other_inc, cv = 10, n_jobs = -1, verbose = 2)

rf_grid_cons_other_inc.fit(X_cons_other_inc_train, y_cons_other_inc_train)

rf_grid_cons_other_inc.best_params_ # This returns the best parameters for use in the final model

# **Model Development (Replication)** #

This section contains the code for the development of the RF model for repair costs. Essentially, it is a replication of the work by Altringer et al (2021)

## Repair Costs ##

In [ ]:
## Final model trained on final parameters

rf_final_rep_repair_inc = RandomForestRegressor(n_estimators=1900,
                                 criterion='squared_error',
                                 max_depth=40,
                                 max_features='sqrt',
                                 min_samples_leaf=2,
                                 min_samples_split=6,
                                 bootstrap=False,
                                 random_state=23)

rf_final_rep_repair_inc.fit(X_rep_repair_inc_train, y_rep_repair_inc_train)

y_pred_rep_repair_inc = rf_final_rep_repair_inc.predict(X_rep_repair_inc_test) # This will 'predict' the repair costs for our test sample and allow us to compare

## Tabulate the results of the model prediction against the

df_results_rep_repair_inc = pd.DataFrame() # Blank dataframe for our test results

df_results_rep_repair_inc['log_costs'] = y_rep_repair_inc_test
df_results_rep_repair_inc['log_predictions'] = y_pred_rep_repair_inc
df_results_rep_repair_inc['log_error'] = df_results_rep_repair_inc['log_predictions'] - df_results_rep_repair_inc['log_costs']
df_results_rep_repair_inc['sq_log_error'] = df_results_rep_repair_inc['log_error'] ** 2

df_results_rep_repair_inc['costs'] = np.exp(1) ** y_rep_repair_inc_test
df_results_rep_repair_inc['predictions'] = np.exp(1) ** y_pred_rep_repair_inc
df_results_rep_repair_inc['error'] = df_results_rep_repair_inc['predictions'] - df_results_rep_repair_inc['costs']
df_results_rep_repair_inc['sq_error'] = df_results_rep_repair_inc['error'] ** 2

### Tabulation of stratified test result calculations ###

df_strat_results_rep_repair_inc = pd.DataFrame(columns=['mse', 'mae', 'r_2'])

n_sample = int(len(df_results_rep_repair_inc) * 0.65)

for i in range(0, 100):
  random_records = random.sample(list(df_results_rep_repair_inc.index), n_sample)
  df_sample = df_results_rep_repair_inc[df_results_rep_repair_inc.index.isin(random_records)]

  temp_mse = df_sample['sq_log_error'].mean()
  temp_mae = abs(df_sample['log_error']).mean()
  temp_r_2 = r2_score(df_sample['log_costs'], df_sample['log_predictions'])

  new_row = {'mse': temp_mse, 'mae': temp_mae, 'r_2': temp_r_2}

  df_strat_results_rep_repair_inc = df_strat_results_rep_repair_inc.append(new_row, ignore_index=True)

total_mse_mean = df_strat_results_rep_repair_inc['mse'].mean()
total_mse_sd = df_strat_results_rep_repair_inc['mse'].std()
total_mae_mean = df_strat_results_rep_repair_inc['mae'].mean()
total_mae_sd = df_strat_results_rep_repair_inc['mae'].std()
total_r_2_mean = df_strat_results_rep_repair_inc['r_2'].mean()
total_r_2_sd = df_strat_results_rep_repair_inc['r_2'].std()

print('Final Results - Replication - Repair Costs')
print('MSE Mean (SD): ' + str(np.round(total_mse_mean, 3)) + ' (' + str(np.round(total_mse_sd, 3)) + ')')
print('MAE Mean (SD): ' + str(np.round(total_mae_mean, 3)) + ' (' + str(np.round(total_mae_sd, 3)) + ')')
print('R2 Mean (SD): ' + str(np.round(total_r_2_mean, 3)) + ' (' + str(np.round(total_r_2_sd, 3)) + ')')

## Other Costs ##

In [ ]:
## Final model trained on final parameters

rf_final_rep_other_inc = RandomForestRegressor(n_estimators=1700,
                                 criterion='squared_error',
                                 max_depth=150,
                                 max_features='sqrt',
                                 min_samples_leaf=2,
                                 min_samples_split=8,
                                 bootstrap=False,
                                 random_state=23)

rf_final_rep_other_inc.fit(X_rep_other_inc_train, y_rep_other_inc_train)

y_pred_rep_other_inc = rf_final_rep_other_inc.predict(X_rep_other_inc_test) # This will 'predict' the repair costs for our test sample and allow us to compare


## Tabulate the results of the model prediction against the

df_results_rep_other_inc = pd.DataFrame() # Blank dataframe for our test results

df_results_rep_other_inc['log_costs'] = y_rep_other_inc_test
df_results_rep_other_inc['log_predictions'] = y_pred_rep_other_inc
df_results_rep_other_inc['log_error'] = df_results_rep_other_inc['log_predictions'] - df_results_rep_other_inc['log_costs']
df_results_rep_other_inc['sq_log_error'] = df_results_rep_other_inc['log_error'] ** 2

df_results_rep_other_inc['costs'] = np.exp(1) ** y_rep_other_inc_test
df_results_rep_other_inc['predictions'] = np.exp(1) ** y_pred_rep_other_inc
df_results_rep_other_inc['error'] = df_results_rep_other_inc['predictions'] - df_results_rep_other_inc['costs']
df_results_rep_other_inc['sq_error'] = df_results_rep_other_inc['error'] ** 2


### Tabulation of stratified test result calculations ###

df_strat_results_rep_other_inc = pd.DataFrame(columns=['mse', 'mae', 'r_2'])

n_sample = int(len(df_results_rep_other_inc) * 0.65)

for i in range(0, 100):
  random_records = random.sample(list(df_results_rep_other_inc.index), n_sample)
  df_sample = df_results_rep_other_inc[df_results_rep_other_inc.index.isin(random_records)]

  temp_mse = df_sample['sq_log_error'].mean()
  temp_mae = abs(df_sample['log_error']).mean()
  temp_r_2 = r2_score(df_sample['log_costs'], df_sample['log_predictions'])

  new_row = {'mse': temp_mse, 'mae': temp_mae, 'r_2': temp_r_2}

  df_strat_results_rep_other_inc = df_strat_results_rep_other_inc.append(new_row, ignore_index=True)

total_mse_mean = df_strat_results_rep_other_inc['mse'].mean()
total_mse_sd = df_strat_results_rep_other_inc['mse'].std()
total_mae_mean = df_strat_results_rep_other_inc['mae'].mean()
total_mae_sd = df_strat_results_rep_other_inc['mae'].std()
total_r_2_mean = df_strat_results_rep_other_inc['r_2'].mean()
total_r_2_sd = df_strat_results_rep_other_inc['r_2'].std()

print('Final Results - Replication - Other Costs')
print('MSE Mean (SD): ' + str(np.round(total_mse_mean, 3)) + ' (' + str(np.round(total_mse_sd, 3)) + ')')
print('MAE Mean (SD): ' + str(np.round(total_mae_mean, 3)) + ' (' + str(np.round(total_mae_sd, 3)) + ')')
print('R2 Mean (SD): ' + str(np.round(total_r_2_mean, 3)) + ' (' + str(np.round(total_r_2_sd, 3)) + ')')

# **Constrained US Model (based on AUS data structure)** #

## Repair Costs ##

In [ ]:
## Final model trained on final parameters

rf_final_cons_repair_inc = RandomForestRegressor(n_estimators=600,
                                 criterion='squared_error',
                                 max_depth=40,
                                 max_features='sqrt',
                                 min_samples_leaf=2,
                                 min_samples_split=5,
                                 bootstrap=False,
                                 random_state=23)

rf_final_cons_repair_inc.fit(X_cons_repair_inc_train, y_cons_repair_inc_train)

y_pred_cons_repair_inc = rf_final_cons_repair_inc.predict(X_cons_repair_inc_test) # This will 'predict' the costs for our test sample and allow us to compare

## Tabulate the results of the model prediction against the

df_results_cons_repair_inc = pd.DataFrame() # Blank dataframe for our test results

df_results_cons_repair_inc['log_costs'] = y_cons_repair_inc_test
df_results_cons_repair_inc['log_predictions'] = y_pred_cons_repair_inc
df_results_cons_repair_inc['log_error'] = df_results_cons_repair_inc['log_predictions'] - df_results_cons_repair_inc['log_costs']
df_results_cons_repair_inc['sq_log_error'] = df_results_cons_repair_inc['log_error'] ** 2

df_results_cons_repair_inc['costs'] = np.exp(1) ** y_cons_repair_inc_test
df_results_cons_repair_inc['predictions'] = np.exp(1) ** y_pred_cons_repair_inc
df_results_cons_repair_inc['error'] = df_results_cons_repair_inc['predictions'] - df_results_cons_repair_inc['costs']
df_results_cons_repair_inc['sq_error'] = df_results_cons_repair_inc['error'] ** 2

### Tabulation of stratified test result calculations ###

df_strat_results_cons_repair_inc = pd.DataFrame(columns=['mse', 'mae', 'r_2'])

n_sample = int(len(df_results_cons_repair_inc) * 0.65)

for i in range(0, 100):
  random_records = random.sample(list(df_results_cons_repair_inc.index), n_sample)
  df_sample = df_results_cons_repair_inc[df_results_cons_repair_inc.index.isin(random_records)]

  temp_mse = df_sample['sq_log_error'].mean()
  temp_mae = abs(df_sample['log_error']).mean()
  temp_r_2 = r2_score(df_sample['log_costs'], df_sample['log_predictions'])

  new_row = {'mse': temp_mse, 'mae': temp_mae, 'r_2': temp_r_2}

  df_strat_results_cons_repair_inc = df_strat_results_cons_repair_inc.append(new_row, ignore_index=True)

total_mse_mean = df_strat_results_cons_repair_inc['mse'].mean()
total_mse_sd = df_strat_results_cons_repair_inc['mse'].std()
total_mae_mean = df_strat_results_cons_repair_inc['mae'].mean()
total_mae_sd = df_strat_results_cons_repair_inc['mae'].std()
total_r_2_mean = df_strat_results_cons_repair_inc['r_2'].mean()
total_r_2_sd = df_strat_results_cons_repair_inc['r_2'].std()

print('Final Results - Constrained - Repair Costs')
print('MSE Mean (SD): ' + str(np.round(total_mse_mean, 3)) + ' (' + str(np.round(total_mse_sd, 3)) + ')')
print('MAE Mean (SD): ' + str(np.round(total_mae_mean, 3)) + ' (' + str(np.round(total_mae_sd, 3)) + ')')
print('R2 Mean (SD): ' + str(np.round(total_r_2_mean, 3)) + ' (' + str(np.round(total_r_2_sd, 3)) + ')')

## Other Costs ##

In [ ]:
## Final model trained on final parameters

rf_final_cons_other_inc = RandomForestRegressor(n_estimators=900,
                                 criterion='squared_error',
                                 max_depth=170,
                                 max_features='sqrt',
                                 min_samples_leaf=4,
                                 min_samples_split=5,
                                 bootstrap=False,
                                 random_state=23)

rf_final_cons_other_inc.fit(X_cons_other_inc_train, y_cons_other_inc_train)

y_pred_cons_other_inc = rf_final_cons_other_inc.predict(X_cons_other_inc_test) # This will 'predict' the repair costs for our test sample and allow us to compare


## Tabulate the results of the model prediction against the

df_results_cons_other_inc = pd.DataFrame() # Blank dataframe for our test results

df_results_cons_other_inc['log_costs'] = y_cons_other_inc_test
df_results_cons_other_inc['log_predictions'] = y_pred_cons_other_inc
df_results_cons_other_inc['log_error'] = df_results_cons_other_inc['log_predictions'] - df_results_cons_other_inc['log_costs']
df_results_cons_other_inc['sq_log_error'] = df_results_cons_other_inc['log_error'] ** 2

df_results_cons_other_inc['costs'] = np.exp(1) ** y_cons_other_inc_test
df_results_cons_other_inc['predictions'] = np.exp(1) ** y_pred_cons_other_inc
df_results_cons_other_inc['error'] = df_results_cons_other_inc['predictions'] - df_results_cons_other_inc['costs']
df_results_cons_other_inc['sq_error'] = df_results_cons_other_inc['error'] ** 2


### Tabulation of stratified test result calculations ###

df_strat_results_cons_other_inc = pd.DataFrame(columns=['mse', 'mae', 'r_2'])

n_sample = int(len(df_results_cons_other_inc) * 0.65)

for i in range(0, 100):
  random_records = random.sample(list(df_results_cons_other_inc.index), n_sample)
  df_sample = df_results_cons_other_inc[df_results_cons_other_inc.index.isin(random_records)]

  temp_mse = df_sample['sq_log_error'].mean()
  temp_mae = abs(df_sample['log_error']).mean()
  temp_r_2 = r2_score(df_sample['log_costs'], df_sample['log_predictions'])

  new_row = {'mse': temp_mse, 'mae': temp_mae, 'r_2': temp_r_2}

  df_strat_results_cons_other_inc = df_strat_results_cons_other_inc.append(new_row, ignore_index=True)

total_mse_mean = df_strat_results_cons_other_inc['mse'].mean()
total_mse_sd = df_strat_results_cons_other_inc['mse'].std()
total_mae_mean = df_strat_results_cons_other_inc['mae'].mean()
total_mae_sd = df_strat_results_cons_other_inc['mae'].std()
total_r_2_mean = df_strat_results_cons_other_inc['r_2'].mean()
total_r_2_sd = df_strat_results_cons_other_inc['r_2'].std()

print('Final Results - Constrained - Other Costs')
print('MSE Mean (SD): ' + str(np.round(total_mse_mean, 3)) + ' (' + str(np.round(total_mse_sd, 3)) + ')')
print('MAE Mean (SD): ' + str(np.round(total_mae_mean, 3)) + ' (' + str(np.round(total_mae_sd, 3)) + ')')
print('R2 Mean (SD): ' + str(np.round(total_r_2_mean, 3)) + ' (' + str(np.round(total_r_2_sd, 3)) + ')')

In [ ]:
## Calculation of significance scores for performance difference

import scipy.stats as stats

# Repair Costs
print('Repair MSE: p-value', round(stats.ttest_ind(a=df_strat_results_rep_repair_inc['mse'], b=df_strat_results_cons_repair_inc['mse'], equal_var=True)[1], 3))
print('Repair MAE: p-value', round(stats.ttest_ind(a=df_strat_results_rep_repair_inc['mae'], b=df_strat_results_cons_repair_inc['mae'], equal_var=True)[1], 3))
print('Repair R^2: p-value', round(stats.ttest_ind(a=df_strat_results_rep_repair_inc['r_2'], b=df_strat_results_cons_repair_inc['r_2'], equal_var=True)[1], 3))

# Other Costs
print('Other MSE: p-value', round(stats.ttest_ind(a=df_strat_results_rep_other_inc['mse'], b=df_strat_results_cons_other_inc['mse'], equal_var=True)[1], 3))
print('Other MAE: p-value', round(stats.ttest_ind(a=df_strat_results_rep_other_inc['mae'], b=df_strat_results_cons_other_inc['mae'], equal_var=True)[1], 3))
print('Other R^2: p-value', round(stats.ttest_ind(a=df_strat_results_rep_other_inc['r_2'], b=df_strat_results_cons_other_inc['r_2'], equal_var=True)[1], 3))

# **Visualisation of Results** #

In [ ]:
sns.set_theme(style="whitegrid", palette="pastel")

fig, axes = plt.subplots(1, 3, figsize=(25, 7))
fig.suptitle('Repair Cost Model Performance', fontsize = 20)

x000 = df_strat_results_rep_repair_inc['mse']
x001 = df_strat_results_cons_repair_inc['mse']

x100 = df_strat_results_rep_repair_inc['mae']
x101 = df_strat_results_cons_repair_inc['mae']

x200 = df_strat_results_rep_repair_inc['r_2']
x201 = df_strat_results_cons_repair_inc['r_2']

sns.kdeplot(ax=axes[0], x=x000, shade=True, color='darkorange')
sns.kdeplot(ax=axes[0], x=x001, shade=True, color='blue')
axes[0].set_xlabel('Mean Square Error', fontsize = 15)
axes[0].set_ylabel('', fontsize = 20)
axes[0].axvline(2.637, color="black", ls="--")
axes[0].set(yticklabels=[])

sns.kdeplot(ax=axes[1], x=x100, shade=True, color='darkorange')
sns.kdeplot(ax=axes[1], x=x101, shade=True, color='blue')
axes[1].set_xlabel('Mean Absolute Error', fontsize = 15)
axes[1].set_ylabel('', fontsize = 20)
axes[1].axvline(1.244, color="black", ls="--")
axes[1].set(yticklabels=[])


sns.kdeplot(ax=axes[2], x=x200, shade=True, color='darkorange')
sns.kdeplot(ax=axes[2], x=x201, shade=True, color='blue')
axes[2].set_xlabel('R-squared', fontsize = 15)
axes[2].set_ylabel('', fontsize = 20)
axes[2].axvline(0.504, color="black", ls="--")
axes[2].set(yticklabels=[])


plt.legend(labels=['Altringer et al Results', 'Full Feature Set', 'Constrained Feature Set'])


In [ ]:
sns.set_theme(style="whitegrid", palette="pastel")

fig, axes = plt.subplots(1, 3, figsize=(25, 7))
fig.suptitle('Other Cost Model Performance', fontsize = 20)

x002 = df_strat_results_rep_other_inc['mse']
x003 = df_strat_results_cons_other_inc['mse']

x102 = df_strat_results_rep_other_inc['mae']
x103 = df_strat_results_cons_other_inc['mae']

x202 = df_strat_results_rep_other_inc['r_2']
x203 = df_strat_results_cons_other_inc['r_2']

sns.kdeplot(ax=axes[0], x=x002, shade=True, color='darkorange')
sns.kdeplot(ax=axes[0], x=x003, shade=True, color='blue')
axes[0].set_xlabel('Mean Square Error', fontsize = 15)
axes[0].set_ylabel('', fontsize = 20)
axes[0].axvline(1.822, color="black", ls="--")
axes[0].set(yticklabels=[])

sns.kdeplot(ax=axes[1], x=x102, shade=True, color='darkorange')
sns.kdeplot(ax=axes[1], x=x103, shade=True, color='blue')
axes[1].set_xlabel('Mean Absolute Error', fontsize = 15)
axes[1].set_ylabel('', fontsize = 20)
axes[1].axvline(0.838, color="black", ls="--")
axes[1].set(yticklabels=[])

sns.kdeplot(ax=axes[2], x=x202, shade=True, color='darkorange')
sns.kdeplot(ax=axes[2], x=x203, shade=True, color='blue')
axes[2].set_xlabel('R-squared', fontsize = 15)
axes[2].set_ylabel('', fontsize = 20)
axes[2].axvline(0.945, color="black", ls="--")
axes[2].set(yticklabels=[])


plt.legend(labels=['Altringer et al Results', 'Full Feature Set', 'Constrained Feature Set'])


# **Application to AUS Data** #

In [ ]:
## The first step is to filter for reports indicating damage

l_damage_indicators = ['M', 'M?', 'S', 'D']
df_aus_ml_damaged_only = df_aus_ml_data[df_aus_ml_data['damage_level'].isin(l_damage_indicators)]

## And now we need dummy variables for all of our categorical data

X_aus_features_dmg_only = pd.get_dummies(df_aus_ml_damaged_only, prefix_sep='_')
X_aus_features_all = pd.get_dummies(df_aus_ml_data, prefix_sep='_')

## And finally we need to drop some columns for data that doesn't appear in the US data

X_aus_features_dmg_only = X_aus_features_dmg_only.drop(['year', 'acft_class_J', 'acft_class_R', 'type_eng_Y'], axis=1)
X_aus_features_all = X_aus_features_all.drop(['year', 'acft_class_J', 'acft_class_R', 'type_eng_C', 'type_eng_Y'], axis=1)


## Repair Costs ##

In [ ]:
## Final model trained on final parameters

rf_final_aus_repair_inc = RandomForestRegressor(n_estimators=600,
                                 criterion='squared_error',
                                 max_depth=40,
                                 max_features='sqrt',
                                 min_samples_leaf=2,
                                 min_samples_split=5,
                                 bootstrap=False,
                                 random_state=23)

rf_final_aus_repair_inc.fit(X_cons_repair_inc, y_cons_repair_inc)

aus_pred_repair_inc = rf_final_aus_repair_inc.predict(X_aus_features_dmg_only)

In [ ]:
aus_pred_log_mean = round(aus_pred_log.mean(), 2)
aus_pred_log_std_dev = round(aus_pred_log.std(), 2)
aus_pred_log_min = round(aus_pred_log.min(), 2)
aus_pred_log_median = round(np.median(aus_pred_log), 2)
aus_pred_log_max = round(aus_pred_log.max(), 2)
aus_pred_log_n = len(aus_pred_log)

print('Log mean:', aus_pred_log_mean)
print('Log std dev:', aus_pred_log_std_dev)
print('Log min:', aus_pred_log_min)
print('Log median:', aus_pred_log_median)
print('Log max:', aus_pred_log_max)
print('Log n:', aus_pred_log_n)
print('')

aus_pred_mean = round(aus_pred_raw.mean())
aus_pred_std_dev = round(aus_pred_raw.std())
aus_pred_min = round(aus_pred_raw.min(), 2)
aus_pred_median = round(np.median(aus_pred_raw))
aus_pred_max = round(aus_pred_raw.max())
aus_pred_n = len(aus_pred_raw)

print('Raw mean:', aus_pred_mean)
print('Raw std dev:', aus_pred_std_dev)
print('Raw min:', aus_pred_min)
print('Raw median:', aus_pred_median)
print('Raw max:', aus_pred_max)
print('Raw n:', aus_pred_n)
print('')

aus_pred_total_cost = aus_pred_raw.sum()
aus_pred_avg_annual_cost = round(aus_pred_total_cost/10, 2)

print('Total cost 2008-2017:', aus_pred_total_cost)
print('Average annual cost (US$):', aus_pred_avg_annual_cost)

Log mean: 8.88
Log std dev: 0.77
Log min: 5.9
Log median: 8.9
Log max: 11.71
Log n: 4640

Raw mean: 9707
Raw std dev: 8912
Raw min: 365.27
Raw median: 7343
Raw max: 121960
Raw n: 4640

Total cost 2008-2017: 45040428.306524664
Average annual cost (US$): 4504042.83


## Other Costs ##

In [ ]:
## Final model trained on final parameters

rf_final_aus_other_inc = RandomForestRegressor(n_estimators=900,
                                 criterion='squared_error',
                                 max_depth=170,
                                 max_features='sqrt',
                                 min_samples_leaf=4,
                                 min_samples_split=5,
                                 bootstrap=False,
                                 random_state=23)

rf_final_aus_other_inc.fit(X_cons_other_inc, y_cons_other_inc)

aus_pred_other_inc = rf_final_aus_other_inc.predict(X_aus_features_all)

In [ ]:
aus_pred_other_log = aus_pred_other_inc
aus_pred_other_raw = np.e ** aus_pred_other_inc

aus_pred_other_log_mean = round(aus_pred_other_log.mean())
aus_pred_other_log_std_dev = round(aus_pred_other_log.std())
aus_pred_other_log_min = round(aus_pred_other_log.min(), 2)
aus_pred_other_log_median = round(np.median(aus_pred_other_log))
aus_pred_other_log_max = round(aus_pred_other_log.max())
aus_pred_other_log_n = len(aus_pred_other_log)

print('Log mean:', aus_pred_other_log_mean)
print('Log std dev:', aus_pred_other_log_std_dev)
print('Log min:', aus_pred_other_log_min)
print('Log median:', aus_pred_other_log_median)
print('Log max:', aus_pred_other_log_max)
print('Log n:', aus_pred_other_log_n)
print('')

aus_pred_other_mean = round(aus_pred_other_raw.mean())
aus_pred_other_std_dev = round(aus_pred_other_raw.std())
aus_pred_other_min = round(aus_pred_other_raw.min(), 2)
aus_pred_other_median = round(np.median(aus_pred_other_raw))
aus_pred_other_max = round(aus_pred_other_raw.max())
aus_pred_other_n = len(aus_pred_other_raw)

print('Raw mean:', aus_pred_other_mean)
print('Raw std dev:', aus_pred_other_std_dev)
print('Raw min:', aus_pred_other_min)
print('Raw median:', aus_pred_other_median)
print('Raw max:', aus_pred_other_max)
print('Raw n:', aus_pred_other_n)
print('')

aus_pred_other_total_cost = aus_pred_other_raw.sum()
aus_pred_other_avg_annual_cost = round(aus_pred_other_total_cost/10, 2)

print('Total cost 2008-2017:', aus_pred_other_total_cost)
print('Average annual cost (US$):', aus_pred_other_avg_annual_cost)

## Annualised Trends ##

Exported to CSV for further analysis

In [ ]:
df_aus_predictions = pd.DataFrame()

df_aus_predictions['year'] = df_aus_ml_data['year']

temp_df_aus_pred_repair = pd.DataFrame()
temp_df_aus_pred_repair['repair_cost_predictions'] = aus_pred_raw
temp_df_aus_pred_repair = temp_df_aus_pred_repair.set_index(df_aus_ml_damaged_only.index)

df_aus_predictions['repair_cost_predictions'] = temp_df_aus_pred_repair['repair_cost_predictions']
df_aus_predictions.loc[df_aus_predictions['repair_cost_predictions'].isnull(), 'repair_cost_predictions'] = 0

df_aus_predictions['other_cost_predictions'] = aus_pred_other_raw

df_aus_predictions['total_cost_predictions'] = df_aus_predictions['other_cost_predictions'] + df_aus_predictions['repair_cost_predictions']

df_aus_predictions.loc[df_aus_predictions['repair_cost_predictions'] == 0, 'repair_cost_predictions'] = np.nan

In [ ]:
df_annual_predictions = pd.DataFrame(columns=['year', 'total_strikes', 'total_repair', 'mean_repair', 'median_repair', 'total_other', 'mean_other', 'median_other', 'total_total', 'mean_total', 'median_total'])

l_years = df_aus_predictions['year'].unique().tolist()

for year in l_years:
  temp_data = df_aus_predictions[df_aus_predictions['year'] ==  year]

  temp_strikes = len(temp_data)

  temp_total_repair = temp_data.loc[temp_data['repair_cost_predictions'].notnull(), 'repair_cost_predictions'].sum()
  temp_mean_repair = temp_data.loc[temp_data['repair_cost_predictions'].notnull(), 'repair_cost_predictions'].mean()
  temp_median_repair = np.median(temp_data.loc[temp_data['repair_cost_predictions'].notnull(), 'repair_cost_predictions'])

  temp_total_other = temp_data['other_cost_predictions'].sum()
  temp_mean_other = temp_data['other_cost_predictions'].mean()
  temp_median_other = np.median(temp_data['other_cost_predictions'])

  temp_total_total = temp_data['total_cost_predictions'].sum()
  temp_mean_total = temp_data['total_cost_predictions'].mean()
  temp_median_total = np.median(temp_data['total_cost_predictions'])

  new_row = {'year': year, 'total_strikes': temp_strikes, 'total_repair': temp_total_repair, 'mean_repair': temp_mean_repair,
             'median_repair': temp_median_repair, 'total_other': temp_total_other,
             'mean_other': temp_mean_other, 'median_other': temp_median_other, 'total_total': temp_total_total,
             'mean_total': temp_mean_total, 'median_total': temp_median_total}

  df_annual_predictions = df_annual_predictions.append(new_row, ignore_index=True)

df_annual_predictions['year'] = df_annual_predictions['year'].astype(int)

In [ ]:
df_annual_predictions.to_csv('final_aus_predictions.csv')